In [1]:
import numpy as np
import pandas as pd 
import plotly.express as px
import json 
import os

df_f = pd.read_csv('../data/farm/far_data_2024-02-21.csv').fillna(0.0)
df_f.head()

,timestamp,cpu_system,boottime,Pool Size Time_P1,mem_free,Missed Buffers_P1,bytes_out,cpu_user,cpu_idle,Pool Size Data_P1,...,RetransSegs_rate,TCPLostRetransmit_rate,TCPForwardRetrans_rate,TotalRetrans,TCPSlowStartRetrans,RPCRetrans,TCPFastRetrans,TCPLostRetransmit,TCPForwardRetrans,RetransSegs
0,2024-02-21 05:36:00,2.100000,0.000000e+00,0.000000,0.0,0.0,0.0,1.006667,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-02-21 05:36:15,2.100000,0.000000e+00,2038.710000,0.0,0.0,0.0,1.300000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-02-21 05:36:30,1.993333,0.000000e+00,2055.490000,0.0,0.0,0.0,1.300000,0.0,11795.033333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-02-21 05:36:45,1.806667,0.000000e+00,2049.026000,0.0,0.0,0.0,1.300000,0.0,11892.133333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-02-21 05:37:00,2.100000,1.705514e+09,2014.643333,0.0,0.0,0.0,1.110000,0.0,11854.600000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Dimensionality Reduction (DR) across time domain, feature domain

In [5]:
from pygam import GAM, s
from scipy.interpolate import BSpline, splrep, splev
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def preprocess(df, value_column):
    return df.loc[:, ['timestamp', 'nodeId', value_column]] \
             .pivot_table(index='timestamp', columns='nodeId', values=value_column) \
             .apply(lambda row: row.fillna(0.0), axis=0).T

# DR1 - applying PCA to each variable across the time domain
def apply_pca_to_feature(col_name, df):
    try:
        # pivot: rows -> timestamps, columns -> nodeId
        X = preprocess(df, col_name)

        X.columns = pd.to_datetime(X.columns)

        start_index = int(len(X.columns) * 0.3)
        end_index = int(len(X.columns) * 0.45)
        X_filtered = X.iloc[:, start_index:end_index]

        # convert to cupy array
        # baseline = cp.array(X.values)
        baseline = X_filtered.values

        # normalizing the data (demean)
        mean_hat = baseline.mean(axis=0)
        demeaned = baseline - mean_hat

        # standardize
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(demeaned)

        if (X_scaled.shape[0] < 2 or np.all(np.isnan(X_scaled)) or np.all(X_scaled == 0)):
            print(f"Skipping {col_name} due to insufficient data variance.")
            return None

        # apply PCA
        pca = PCA(n_components=1) # look into n_components in PCA sklearn implementation
        scores = pca.fit_transform(X_scaled)

        explained_variance_ratio_cumsum = np.cumsum(pca.explained_variance_ratio_)
        npc = np.sum(explained_variance_ratio_cumsum < 0.9999) + 1
        
        abs_comp = np.abs(pca.components_[0])

        top_10 = np.argsort(abs_comp)[-10:][::-1]

        # most influential timestamp for this column
        fc_t = X.columns[top_10] 
        # print(fc_t, 'yo')

        P_fin = pd.DataFrame({f"PC{k+1}": scores[:, k] if k < npc else np.nan for k in range(1)})
        P_fin['Measurement'] = X.index
        
        fc_t_df = pd.DataFrame({'feature': col_name, 'timestamp': fc_t})

        # print(f"{col_name} done...")

        return P_fin, fc_t_df
    
    except Exception as e:
        print(f"Error processing {col_name}: {e}")
        return None

In [15]:
numeric_cols = df_f.drop(columns=['timestamp', 'nodeId']).columns
len(numeric_cols)

46

In [6]:
from concurrent.futures import ThreadPoolExecutor

numeric_cols = df_f.drop(columns=['timestamp', 'nodeId']).columns
P_final = []
FC_final = []

def process_column(col):
    try:
        P_df, fc_t_df = apply_pca_to_feature(col, df_f)
        if P_df is not None:
            P_df.insert(0, 'Col', col)
            P_final.append(P_df)

        if fc_t_df is not None:
            FC_final.append(fc_t_df)

    except Exception as e:
        print(f"Error processing {col}: {e}")
        return None

with ThreadPoolExecutor() as executor:
    executor.map(process_column, numeric_cols)

P_final = pd.concat(P_final, ignore_index=True) if P_final else pd.DataFrame()
FC_final = pd.concat(FC_final, ignore_index=True) if FC_final else pd.DataFrame()

Skipping mem_shared due to insufficient data variance.
Error processing mem_shared: cannot unpack non-iterable NoneType object
Skipping Missed Buffers_P1 due to insufficient data variance.
Error processing Missed Buffers_P1: cannot unpack non-iterable NoneType object
Skipping TotalRetrans_rate due to insufficient data variance.
Error processing TotalRetrans_rate: cannot unpack non-iterable NoneType object
Skipping RPCRetrans_rate due to insufficient data variance.
Error processing RPCRetrans_rate: cannot unpack non-iterable NoneType object
Skipping TCPFastRetrans_rate due to insufficient data variance.
Error processing TCPFastRetrans_rate: cannot unpack non-iterable NoneType object
Skipping TCPSlowStartRetrans_rate due to insufficient data variance.
Error processing TCPSlowStartRetrans_rate: cannot unpack non-iterable NoneType object
Skipping RetransSegs_rate due to insufficient data variance.
Error processing RetransSegs_rate: cannot unpack non-iterable NoneType object
Skipping TCPLos

In [7]:
P_final

,Col,PC1,Measurement
0,load_fifteen,32.804654,novadaq-far-farm-06
1,load_fifteen,33.034022,novadaq-far-farm-07
2,load_fifteen,33.195171,novadaq-far-farm-08
3,load_fifteen,33.143595,novadaq-far-farm-09
4,load_fifteen,33.109554,novadaq-far-farm-10
...,...,...,...
5845,cpu_num,8.322923,novadaq-far-farm-95
5846,cpu_num,-9.164389,novadaq-far-farm-96
5847,cpu_num,-27.804271,novadaq-far-farm-97
5848,cpu_num,8.322923,novadaq-far-farm-98


In [8]:
FC_final.head(20)

,feature,timestamp
0,load_fifteen,2024-02-21 06:56:30
1,load_fifteen,2024-02-21 06:53:00
2,load_fifteen,2024-02-21 06:54:15
3,load_fifteen,2024-02-21 06:54:30
4,load_fifteen,2024-02-21 06:56:15
5,load_fifteen,2024-02-21 06:52:45
6,load_fifteen,2024-02-21 06:55:45
7,load_fifteen,2024-02-21 06:57:45
8,load_fifteen,2024-02-21 06:57:00
9,load_fifteen,2024-02-21 06:52:30


In [18]:
FC_final.to_csv('FC_final.csv')

In [9]:
df_f.loc[:, ['timestamp', 'nodeId', 'Missed Buffers_P1']] \
             .pivot_table(index='timestamp', columns='nodeId', values='Missed Buffers_P1') \
             .apply(lambda row: row.fillna(0.0), axis=0).T

timestamp,2024-02-21 05:36:00,2024-02-21 05:36:15,2024-02-21 05:36:30,2024-02-21 05:36:45,2024-02-21 05:37:00,2024-02-21 05:37:15,2024-02-21 05:37:30,2024-02-21 05:37:45,2024-02-21 05:38:00,2024-02-21 05:38:15,...,2024-02-21 23:57:30,2024-02-21 23:57:45,2024-02-21 23:58:00,2024-02-21 23:58:15,2024-02-21 23:58:30,2024-02-21 23:58:45,2024-02-21 23:59:00,2024-02-21 23:59:15,2024-02-21 23:59:30,2024-02-21 23:59:45
nodeId,,,,,,,,,,,,,,,,,,,,,
novadaq-far-farm-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
novadaq-far-farm-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
novadaq-far-farm-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
novadaq-far-farm-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
novadaq-far-farm-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
novadaq-far-farm-95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
novadaq-far-farm-96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
novadaq-far-farm-97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Below only works with entire dataset (Missed Buffers_P1 is 0 for most of the day)

In [10]:
P_test = apply_pca_to_feature('Missed Buffers_P1', df_f)
P_test

Skipping Missed Buffers_P1 due to insufficient data variance.


In [12]:
P_final

,Col,PC1,Measurement
0,load_fifteen,32.804654,novadaq-far-farm-06
1,load_fifteen,33.034022,novadaq-far-farm-07
2,load_fifteen,33.195171,novadaq-far-farm-08
3,load_fifteen,33.143595,novadaq-far-farm-09
4,load_fifteen,33.109554,novadaq-far-farm-10
...,...,...,...
5845,cpu_num,8.322923,novadaq-far-farm-95
5846,cpu_num,-9.164389,novadaq-far-farm-96
5847,cpu_num,-27.804271,novadaq-far-farm-97
5848,cpu_num,8.322923,novadaq-far-farm-98


In [13]:
P_final['Col'].nunique()

30

In [19]:
P_final = P_final.drop(columns=['y'])
P_final

,Col,PC1,FC_t,Measurement
0,cpu_system,-102.807111,2024-02-21 20:56:30,novadaq-far-farm-06
1,cpu_system,-103.581607,2024-02-21 20:56:30,novadaq-far-farm-07
2,cpu_system,-110.024682,2024-02-21 20:56:30,novadaq-far-farm-08
3,cpu_system,-103.603792,2024-02-21 20:56:30,novadaq-far-farm-09
4,cpu_system,-104.891379,2024-02-21 20:56:30,novadaq-far-farm-10
...,...,...,...,...
6040,cpu_num,-1.654641,2024-02-21 21:06:00,novadaq-far-farm-95
6041,cpu_num,-21.738115,2024-02-21 21:06:00,novadaq-far-farm-96
6042,cpu_num,-33.847232,2024-02-21 21:06:00,novadaq-far-farm-97
6043,cpu_num,-3.033610,2024-02-21 21:06:00,novadaq-far-farm-98


In [14]:
P_final.to_csv('P_final1.csv')

Second step DR: UMAP

In [15]:
from umap import UMAP
import plotly.express as px

# two-step DR: PCA+UMAP
df_pivot = P_final.pivot(index="Measurement", columns="Col", values="PC1")
reducer = UMAP(n_components=2, random_state=42)
embedding = reducer.fit_transform(df_pivot)
df_pivot['UMAP1'] = embedding[:, 0]
df_pivot['UMAP2'] = embedding[:, 1]
df_pivot['nodeId'] = df_pivot.index 
fig = px.scatter(df_pivot, x='UMAP1', y='UMAP2', hover_data={'nodeId': True})

fig.update_layout(title="UMAP Projection",
                  xaxis_title="UMAP 1",
                  yaxis_title="UMAP 2")
fig.show()

/Users/allison/anaconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/allison/anaconda3/envs/myenv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Second step DR: tSNE

In [16]:
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE

# two-step DR: PCA+tSNE
df_tsne = df_pivot.drop(columns=['UMAP1', 'UMAP2', 'nodeId'])
tsne = TSNE(n_components=2, random_state=42)
embedding = tsne.fit_transform(df_tsne)

df_pivot['tSNE1'] = embedding[:, 0]
df_pivot['tSNE2'] = embedding[:, 1]

fig = px.scatter(df_pivot, x='tSNE1', y='tSNE2', hover_data={'nodeId': True})

fig.update_layout(title="t-SNE Projection",
                  xaxis_title="t-SNE 1",
                  yaxis_title="t-SNE 2")

fig.show()


/Users/allison/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/allison/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



Second step DR: PCA

In [61]:
import plotly.express as px
import pandas as pd

# DR2 - applying PCA to each feature across one node/component 
def apply_pca(X, n_components=2):
    try:
        baseline = X.values

        # normalizing the data (demean)
        mean_hat = baseline.mean(axis=0)
        demeaned = baseline - mean_hat

        # standardize
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(demeaned)

        # apply PCA
        pca = PCA(n_components=n_components)
        scores = pca.fit_transform(X_scaled)

        explained_variance_ratio_cumsum = np.cumsum(pca.explained_variance_ratio_)
        npc = np.sum(explained_variance_ratio_cumsum < 0.9999) + 1
        n_components = scores.shape[1]
        print(f"Number of principal components: {n_components}")
        

        P_fin = pd.DataFrame({f"PC{k+1}": scores[:, k] if k < n_components else np.nan for k in range(3)})
        P_fin['Measurement'] = X.index

        return P_fin

    except Exception as e:
        print(f"Error processing PCA across features: {e}")
        return None

In [62]:
df_pca = df_pivot.drop(columns=['UMAP1', 'UMAP2', 'tSNE1', 'tSNE2', 'nodeId'])
P_df = apply_pca(df_pca)

Number of principal components: 2


In [63]:
P_df

,PC1,PC2,PC3,Measurement
0,14.070342,-2.252934,NaN,novadaq-far-farm-06
1,13.898073,-2.905247,NaN,novadaq-far-farm-07
2,14.296000,-4.006608,NaN,novadaq-far-farm-08
3,14.216686,-2.622160,NaN,novadaq-far-farm-09
4,13.550015,-2.195082,NaN,novadaq-far-farm-10
...,...,...,...,...
190,-1.859436,1.464636,NaN,novadaq-far-farm-95
191,1.183160,-0.081462,NaN,novadaq-far-farm-96
192,2.327326,-0.884678,NaN,novadaq-far-farm-97
193,-2.028562,0.979259,NaN,novadaq-far-farm-98


In [64]:
P_df.set_index('Measurement', inplace=True)
df_pca['PC1'] = P_df['PC1']
df_pca['PC2'] = P_df['PC2']
df_pca['nodeId'] = df_pca.index
df_pca

Col,Missed Buffers_P1,Pool Size Data_P1,Pool Size Events_P1,Pool Size Time_P1,boottime,bytes_in,bytes_out,cpu_aidle,cpu_idle,cpu_nice,...,part_max_used,pkts_in,pkts_out,proc_run,proc_total,swap_free,swap_total,PC1,PC2,nodeId
Measurement,,,,,,,,,,,,,,,,,,,,,
novadaq-far-farm-06,-24.256130,221.502564,223.920172,223.214097,-22.035973,204.857742,-134.147774,16.808551,127.056379,113.733076,...,5.687101,191.667016,222.143552,97.883786,-70.687349,-7.153418,-7.165049,14.070342,-2.252934,novadaq-far-farm-06
novadaq-far-farm-07,-24.256130,221.502564,223.920172,223.214097,-20.147498,204.986937,-134.194588,-14.473576,125.153507,114.064763,...,-29.307690,192.456964,222.238555,95.922728,-70.837992,-7.153418,-7.165049,13.898073,-2.905247,novadaq-far-farm-07
novadaq-far-farm-08,-24.256130,221.502564,223.920172,223.214097,22.105131,205.294531,-134.363631,-20.789340,75.072762,113.736503,...,24.567064,201.005329,222.180963,98.238420,-86.532456,-7.153418,-7.165049,14.296000,-4.006608,novadaq-far-farm-08
novadaq-far-farm-09,-24.256130,221.502564,223.920172,223.214097,-11.154821,205.093293,-134.319299,11.106108,115.991741,113.735090,...,10.276417,193.845055,222.321312,98.292996,-73.277183,-7.153418,-7.165049,14.216686,-2.622160,novadaq-far-farm-09
novadaq-far-farm-10,-24.256130,221.502564,223.920172,223.214097,-16.779916,204.977334,-134.248648,-15.117958,122.038137,113.735767,...,28.671451,192.887712,222.281727,97.878586,-71.640847,-7.153418,-7.165049,13.550015,-2.195082,novadaq-far-farm-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
novadaq-far-farm-95,-24.256130,-26.012181,-25.092464,-25.399800,11.557186,-27.051852,24.159324,47.763506,14.555998,-23.449652,...,-60.641238,-27.975056,-26.469337,-25.627451,5.713262,-7.153418,-7.165049,-1.859436,1.464636,novadaq-far-farm-95
novadaq-far-farm-96,-24.223770,3.295862,3.401222,3.632307,56.033327,1.961146,-13.428502,25.778116,-11.183348,4.660297,...,-41.043307,5.308201,1.691727,13.884449,-18.087210,-7.153418,-7.165049,1.183160,-0.081462,novadaq-far-farm-96
novadaq-far-farm-97,-22.252491,22.593992,21.756998,22.532210,29.394489,21.438514,-32.353491,-28.771354,-29.200002,23.607357,...,-7.177163,25.577577,20.207151,31.812149,-33.138204,-7.153418,-7.165049,2.327326,-0.884678,novadaq-far-farm-97


In [65]:
fig = px.scatter(df_pca, x="PC1", y="PC2", hover_data={'nodeId': True})

fig.update_layout(title="PCA Projection",
                  xaxis_title="PCA 1",
                  yaxis_title="PCA 2")

fig.show()

In [66]:
df_pivot['PC1'] = df_pca['PC1']
df_pivot['PC2'] = df_pca['PC2']
df_pivot.drop(columns=['nodeId']).to_csv('multiDR_results1.csv')